In [1]:
import os
import numpy as np
from sklearn import svm
from skimage import io, color
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tpot import TPOTClassifier

In [3]:
import os
import cv2
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Definir as dimensões desejadas para redimensionar as imagens
image_width = 256
image_height = 256

# Definir os diretórios das pastas contendo as imagens e uma lista vazia para armazenar as imagens e seus rótulos
data_dir = 'archive/images'
image_folders = sorted(os.listdir(data_dir))
images = []
labels = []

# Função para ler e redimensionar as imagens
def read_and_resize_image(file_path):
    image = cv2.imread(file_path)
    image = cv2.resize(image, (image_width, image_height))
    return image.flatten()  # Retorna o array unidimensional

# Percorrer as pastas e ler as imagens
for folder in image_folders:
    folder_path = os.path.join(data_dir, folder)
    if os.path.isdir(folder_path):
        image_files = sorted(os.listdir(folder_path))
        for image_file in image_files:
            image_path = os.path.join(folder_path, image_file)
            image = read_and_resize_image(image_path)
            images.append(image)
            labels.append(folder) 

# Converter as listas de imagens e rótulos em arrays numpy
images = np.array(images)
labels = np.array(labels)

images, labels = shuffle(images, labels, random_state=42)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
from sklearn.decomposition import PCA

pca = PCA(n_components=10)  # Escolha o número de componentes desejado
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [8]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train_pca, y_train)

y_pred = clf.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 13.19%


In [10]:
tpot = TPOTClassifier(generations=5, population_size=40, verbosity=2, random_state=42,subsample=1,max_eval_time_mins=10)
tpot.fit(X_train_pca, y_train)

                                                                               
Generation 1 - Current best internal CV score: 0.1951190476190476
                                                                                
Generation 2 - Current best internal CV score: 0.1955952380952381
                                                                                 
Generation 3 - Current best internal CV score: 0.20309523809523808
                                                                                
Generation 4 - Current best internal CV score: 0.20309523809523808
                                                                                
Generation 5 - Current best internal CV score: 0.20309523809523808
                                                                                
Best pipeline: MLPClassifier(PCA(MaxAbsScaler(input_matrix), iterated_power=3, svd_solver=randomized), alpha=0.001, learning_rate_init=0.01)


TPOTClassifier(generations=5, max_eval_time_mins=10, population_size=40,
               random_state=42, subsample=1, verbosity=2)

In [11]:
y_pred = tpot.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 21.71%
